In [36]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from skimage.feature import hog, local_binary_pattern
import matplotlib.pyplot as plt

In [37]:
# Fungsi deteksi dan pra-pemrosesan wajah
def detect_and_preprocess_face(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('assets/haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3, minSize=(100, 100))

    if len(faces) > 0:
        x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (100, 100), interpolation=cv2.INTER_AREA)

        # Visualisasi wajah setelah dipotong dan diresize
        # plt.imshow(face, cmap="gray")
        # plt.title("Preprocessed Face")
        # plt.axis("off")
        # plt.show()

        mask = np.zeros_like(face)
        mask = cv2.circle(mask, (50, 50), 40, 255, -1)
        face = cv2.bitwise_and(face, mask)

        # Visualisasi wajah setelah masking
        # plt.imshow(face, cmap="gray")
        # plt.title("Masked Face")
        # plt.axis("off")
        # plt.show()

        return face
    return None

In [38]:
# Fungsi ekstraksi fitur
def extract_canny_features(face):
    edges = cv2.Canny(face, threshold1=30, threshold2=100)

    # Visualisasi hasil deteksi tepi
    # plt.imshow(edges, cmap="gray")
    # plt.title("Canny Edges")
    # plt.axis("off")
    # plt.show()

    edges_hist, _ = np.histogram(edges.ravel(), bins=np.arange(0, 256), density=True)
    return edges_hist
def extract_hog_features(face):
    hog_features, hog_image = hog(face, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                                  block_norm='L2-Hys', visualize=True, feature_vector=True)

    # Visualisasi hasil ekstraksi HOG
    # plt.imshow(hog_image, cmap="gray")
    # plt.title("HOG Features")
    # plt.axis("off")
    # plt.show()

    return hog_features
def extract_lbp_features(face):
    lbp = local_binary_pattern(face, P=8, R=1, method='uniform')

    # Visualisasi pola LBP
    # plt.imshow(lbp, cmap="gray")
    # plt.title("LBP Image")
    # plt.axis("off")
    # plt.show()

    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), density=True)
    return lbp_hist

In [39]:
def extract_combined_features(face):
    canny_features = extract_canny_features(face)
    hog_features = extract_hog_features(face)
    lbp_features = extract_lbp_features(face)
    return np.hstack((canny_features, hog_features, lbp_features))

In [40]:
# Defining a function to return the class labels corresponding to the age-ranges shown above.
def class_labels(age):
    if 1 <= age <= 10:
        return 0
    elif 11 <= age <= 20:
        return 1
    elif 21 <= age <= 30:
        return 2
    elif 31 <= age <= 40:
        return 3
    elif 41 <= age <= 50:
        return 4
    # elif 51 <= age <= 60:
    #     return 5
    # elif 61 <= age <= 70:
    #     return 6
    # elif 71 <= age <= 80:
    #     return 7
    else:
        return 5

In [41]:
label_mapping = {
    '1-10 anak': 0,
    '11-20 remaja': 1,
    '21-30 transisi': 2,
    '31-40 masa matang': 3,
    '41-50 dewasa': 4,
    '51+ usia pertengahan': 5,
    # '61-70 tua': 6,
    # '71-80 lanjut usia': 7,
    # '81-90 lanjut usia tua': 8
}

In [42]:
import os
import cv2
import numpy as np
import logging

def prepare_data(data_dir):
    """
    Fungsi untuk mempersiapkan data pelatihan dari direktori gambar.

    Parameters:
        data_dir (str): Path ke direktori utama data.

    Returns:
        tuple: Dua numpy array, yaitu features dan labels.
    """
    features = []
    labels = []

    if not os.path.exists(data_dir):
        logging.error(f"Direktori {data_dir} tidak ditemukan.")
        return np.array(features), np.array(labels)

    for label_dir in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label_dir)

        if not os.path.isdir(label_path):
            logging.warning(f"{label_path} bukan direktori, dilewati.")
            continue

        label = label_dir  # Menggunakan nama folder sebagai label

        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)

            try:
                image = cv2.imread(img_path)
                if image is None:
                    logging.warning(f"Gagal membaca gambar {img_path}, dilewati.")
                    continue

                # Deteksi dan preprocess wajah
                face = detect_and_preprocess_face(image)
                if face is None:
                    logging.warning(f"Wajah tidak terdeteksi dalam {img_path}, dilewati.")
                    continue

                # Ekstraksi fitur
                feature = extract_combined_features(face)
                features.append(feature)
                labels.append(label)

            except Exception as e:
                logging.error(f"Terjadi error pada file {img_path}: {e}")

    features = np.array(features)
    labels = np.array(labels)

    logging.info(f"Data telah diproses. Total fitur: {features.shape[0]}, Total label: {labels.shape[0]}.")
    return features, labels


In [43]:
# Fungsi pelatihan KNN
def train_knn(X, y, k=5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return knn, X_train, y_train

# Fungsi pengujian model
def test_knn_model(test_image_path, knn, X_train, y_train, label_mapping):
    test_image = cv2.imread(test_image_path)
    face = detect_and_preprocess_face(test_image)
    if face is None:
        print("No face detected in test image.")
        return

    test_feature = extract_combined_features(face)
    distances, indices = knn.kneighbors([test_feature], n_neighbors=3)

    if len(indices[0]) > 0:
        top_matches = []
        for i, idx in enumerate(indices[0]):
            label = y_train[idx]
            label_name = [name for name, id_ in label_mapping.items() if id_ == label][0]
            top_matches.append((label_name, distances[0][i]))

        print("Top-3 Matches:")
        for rank, (label, dist) in enumerate(top_matches, 1):
            print(f"{rank}. Label: {label}, Distance: {dist:.2f}")

        # Visualisasi gambar pengujian
        plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
        plt.title(f"Predicted: {top_matches[0][0]}")
        plt.axis("off")
        plt.show()
    else:
        print("No neighbors found for the test feature.")

In [44]:
if __name__ == "__main__":
    data_dir = r"assets/test/"
    test_image_path = r"cekombak.jpg"

    label_mapping = {"Person1": 0, "Person2": 1, "Person3": 2}  # Sesuaikan dengan dataset Anda
    X, y = prepare_data(data_dir)

    if X is None or y is None or len(X) == 0 or len(y) == 0:
        print("Dataset is empty. Please check the data directory and ensure it contains valid data.")
    else:
        knn, X_train, y_train = train_knn(X, y)
        test_knn_model(test_image_path, knn, X_train, y_train, label_mapping)

Accuracy: 25.00%


IndexError: list index out of range

# SKIP

In [ ]:
# Fungsi pelatihan KNN
def train_knn(X, y, k=5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return knn, X_train, y_train
# Fungsi pengujian model
def test_knn_model(test_image_path, knn, X_train, y_train):
    test_image = cv2.imread(test_image_path)
    face = detect_and_preprocess_face(test_image)
    if face is None:
        print("No face detected in test image.")
        return

    test_feature = extract_combined_features(face)
    distances, indices = knn.kneighbors([test_feature], n_neighbors=3)

    top_matches = []
    for i, idx in enumerate(indices[0]):
        label = y_train[idx]
        label_name = [name for name, id_ in label_mapping.items() if id_ == label][0]
        top_matches.append((label_name, distances[0][i]))

    print("Top-3 Matches:")
    for rank, (label, dist) in enumerate(top_matches, 1):
        print(f"{rank}. Label: {label}, Distance: {dist:.2f}")
 # Visualisasi gambar pengujian
    plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
    plt.title(f"Predicted: {top_matches[0][0]}")
    plt.axis("off")
    plt.show()
# Main script
if __name__ == "__main__":
    data_dir = r"assets/test/"
    test_image_path = r"cekombak.jpg"

    X, y = prepare_data(data_dir)
    if X.size == 0 or y.size == 0:
        print("Dataset is empty. Please check the data directory and ensure it contains valid data.")
    else:
        knn, X_train, y_train = train_knn(X, y)
        test_knn_model(test_image_path, knn, X_train, y_train)

Accuracy: 25.00%


IndexError: list index out of range